In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from urllib.request import urlopen
import csv
import numpy as np

centre = [55.7538445, 37.6200898]
ring1 = [55.750531, 37.642532]
ring2 = [55.730041, 37.612037]
ring3 = [55.709377, 37.583605]
ring4 = [55.638916, 37.459285]

rad1 = np.sqrt((ring1[0] - centre[0])**2 + (ring1[1] - centre[1])**2)
rad2 = np.sqrt((ring2[0] - centre[0])**2 + (ring2[1] - centre[1])**2)
rad3 = np.sqrt((ring3[0] - centre[0])**2 + (ring3[1] - centre[1])**2)
rad4 = np.sqrt((ring4[0] - centre[0])**2 + (ring4[1] - centre[1])**2)

def dist(x, y):
    return np.sqrt((x - centre[0])**2 + (y - centre[1])**2)

def ring(x, y):
    distance = dist(x, y)
    if (distance < rad1):
        return 0
    elif (distance < rad2):
        return 1
    elif (distance < rad3):
        return 2
    elif (distance < rad4):
        return 3
    else:
        return 4

def cosin(first, second):
    AB = 0
    A = 0
    B = 0
    for i in range (len(first)):
        AB += first[i] * second[i]
        A += first[i]**2
        B += second[i]**2
    A = np.sqrt(A)
    B = np.sqrt(B)
    return AB / (A * B)

def from_address_to_ring(address):
    url = "https://www.google.com/maps/search/" + address
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome('/Users/slavapirogov/Downloads/chromedriver', options=chrome_options)
    driver.get(url)
    address = driver.find_element_by_css_selector('meta[itemprop=image]').get_attribute('content')
    driver.close()
    address = address.split('?center=')[1].split('&zoom=')[0].split('%2C')
    ringg = ring(float(address[0]), float(address[1]))
    return ringg

def from_location_to_vec(location):
    address = location[1]
    ring = from_address_to_ring(address) * 4
    
    price = int(location[2]) / 500
    
    floor = int(location[3])
    if (floor == 1):
        floor = 0
    elif (floor <= 6):
        floor = 1
    elif (floor <= 16):
        floor = 2
    elif (floor <= 30):
        floor = 3
    else:
        floor = 4
        
    rooms = (int(location[5]) - 1) * 2
    
    square = int(location[6]) / 10
    
    answ = []
    answ.append(ring)
    answ.append(price)
    answ.append(floor)
    answ.append(rooms)
    answ.append(square)
    return answ

def from_csv_file_to_dict(name_of_file):
    answ = {}
    with open(name_of_file, encoding='utf-8') as file:
        quest = 0
        for row in file:
            if (quest == 0):
                quest = 1
                continue
            location = row.split(";")
            vec = []
            vec.append(location[0])
            vec.append(location[1])
            vec.append(location[2])
            vec.append(location[3])
            vec.append(location[4])
            vec.append(location[5])
            vec.append(location[6])
            vec.append(location[7])
            vec.append(location[8])
            vec = from_location_to_vec(vec)
            answ[location[8]] = vec
    return answ


def from_users_locations_to_vec(users_locations_name_of_file):
    answ = [0] * 5
    #print(answ)
    with open(users_locations_name_of_file, encoding='utf-8') as file:
        quest = 0
        total_locations = 0
        for row in file:
            if (quest == 0):
                quest = 1
                continue
            location = row.split(";")
            total_locations += 1
            vec_of_location = from_location_to_vec(location)
            for i in range(len(answ)):
                answ[i] += vec_of_location[i]
    for i in range(len(answ)):
        answ[i] /= total_locations
    return answ 

def recomendations_for_user(users_locations_name_of_file, locations_name_of_file):
    user_vec = from_users_locations_to_vec(users_locations_name_of_file)
    locations_dict = from_csv_file_to_dict(locations_name_of_file)
    cos_between_user_and_locations = {}
    for key in locations_dict:
        cos_between_user_and_locations[key] = cosin(user_vec, locations_dict[key])
    recomendations_for_user = sorted(cos_between_user_and_locations.items(), key=lambda x: x[1], reverse=True)
    answ_in_list = []
    for key in recomendations_for_user:
        answ_in_list.append(key[0])
    return answ_in_list
        
        
        
        
recomendations_for_user("/Users/slavapirogov/Downloads/locations_3.csv", "/Users/slavapirogov/Downloads/locations.csv")

['https://apartlux.ru/kvartiry/volkov_per_5/\n',
 'https://apartlux.ru/kvartiry/ul_gruzinskii_val_26s1/\n',
 'https://apartlux.ru/kvartiry/ul_zatsepskii_val_4s1/\n',
 'https://apartlux.ru/kvartiry/ul_bolshaia_dorogomilovskaia__d8/\n',
 'https://apartlux.ru/kvartiry/ul_zatsepskii_val_613/\n',
 'https://apartlux.ru/kvartiry/ul_novyi_arbat_10/\n',
 'https://apartlux.ru/kvartiry/prt_andropova_d_37_k2/\n',
 'https://apartlux.ru/kvartiry/ploshchad_pobedy_1b/\n',
 'https://apartlux.ru/kvartiry/priiutskii_pereulok_3/\n',
 'https://apartlux.ru/kvartiry/leninskii_prospekt_7/\n',
 'https://apartlux.ru/kvartiry/ul_1905_goda_zhivova_10/\n',
 'https://apartlux.ru/kvartiry/ploshchad_pobedy_ul_generala_ermolova_2/\n',
 'https://apartlux.ru/kvartiry/ul_bolshaia_dorogomilovskaia_72/\n',
 'https://apartlux.ru/kvartiry/ul_trifonovskaia_61k1/\n',
 'https://apartlux.ru/kvartiry/ul_profsoiuznaia_24k2/\n',
 'https://apartlux.ru/kvartiry/ul_novyi_arbat_26/\n',
 'https://apartlux.ru/kvartiry/prospekt_vernadskog